In [1]:
from sklearn import tree
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer

In [19]:
df = pd.read_csv('data/big_filtered.csv')

df=df.iloc[:, 1:]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8113 entries, 0 to 8112
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ID_                    8113 non-null   int64  
 1   imdb_title_id          8113 non-null   object 
 2   title                  8113 non-null   object 
 3   original_title         8113 non-null   object 
 4   year                   8113 non-null   int64  
 5   date_published         8113 non-null   object 
 6   genre                  8113 non-null   object 
 7   duration               8113 non-null   int64  
 8   country                8112 non-null   object 
 9   language               8088 non-null   object 
 10  director               8112 non-null   object 
 11  writer                 8095 non-null   object 
 12  production_company     8047 non-null   object 
 13  actors                 8111 non-null   object 
 14  description            8066 non-null   object 
 15  avg_

In [4]:
import datetime as dt  #need datetime to extract the month

In [11]:
df1 = df  #making a copy, but this operation seems to have affected the original as well (???)
df1['date_published'] = pd.to_datetime(df['date_published'], unit='D')

In [15]:
df1['month_of_date'] = df1['date_published'].dt.month
df1.head()

,ID_,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,...,metascore,reviews_from_users,reviews_from_critics,intyear,budget_curr,budget_amt,budget_int,ROI_pct,roi_outlier_ind,month_of_date
0,81215,tt6793280,The Devil's Doorway,The Devil's Doorway,2018,2019-02-28,Horror,76,"Ireland, UK",English,...,48.0,48.0,45.0,2018,$,1,1,5.166590e+07,True,2
1,52491,tt10183542,Firincinin Karisi,Firincinin Karisi,2019,2019-09-27,Comedy,91,Turkey,Turkish,...,NaN,NaN,1.0,2019,$,2,2,2.355900e+07,True,9
2,61919,tt1783232,Fetih 1453,Fetih 1453,2012,2012-02-16,"Action, Drama, History",162,Turkey,Turkish,...,NaN,154.0,21.0,2012,$,18,18,1.988724e+08,True,2
3,31463,tt0122143,The Last Broadcast,The Last Broadcast,1998,1998-10-23,"Horror, Mystery",86,USA,English,...,NaN,171.0,46.0,1998,$,900,900,1.244111e+03,True,10
4,54302,tt1127855,Ri-teon,Ri-teon,2007,2007-08-08,"Drama, Thriller",113,South Korea,Korean,...,NaN,1.0,11.0,2007,$,1000,1000,4.568780e+05,True,8


In [23]:
month_series = df1['month_of_date']

In [28]:
dfmth_only = pd.DataFrame(month_series)
dfmth_only= dfmth_only.rename({'month_of_date':'rease_month'})

In [36]:
dfmth_only.to_csv('month_only.csv')

In [35]:
df1['release_mth_str']=df1['month_of_date'].astype(str)
df1['release_mth_str'].value_counts()

10    796
9     767
3     758
4     749
1     703
8     687
11    658
2     658
5     630
6     609
12    578
7     520
Name: release_mth_str, dtype: int64

In [38]:
#convert ints representing months to string because the count vectorizer expects a lower case string

#first I need a mapping
mth_dict={
1:"jan"
,2:"feb"
,3:"mar"
,4:"apr"
,5:"may"
,6:"jun"
,7:"jul"
,8:"aug"
,9:"sep"
,10:"oct"
,11:"nov"
,12:"dec"
}

#second, to join this with month ints
df1['month_abbr'] = df1['month_of_date'].map(mth_dict)
#validation
df1.head()

,ID_,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,...,reviews_from_critics,intyear,budget_curr,budget_amt,budget_int,ROI_pct,roi_outlier_ind,month_of_date,release_mth_str,month_abbr
0,81215,tt6793280,The Devil's Doorway,The Devil's Doorway,2018,2019-02-28,Horror,76,"Ireland, UK",English,...,45.0,2018,$,1,1,5.166590e+07,True,2,2,feb
1,52491,tt10183542,Firincinin Karisi,Firincinin Karisi,2019,2019-09-27,Comedy,91,Turkey,Turkish,...,1.0,2019,$,2,2,2.355900e+07,True,9,9,sep
2,61919,tt1783232,Fetih 1453,Fetih 1453,2012,2012-02-16,"Action, Drama, History",162,Turkey,Turkish,...,21.0,2012,$,18,18,1.988724e+08,True,2,2,feb
3,31463,tt0122143,The Last Broadcast,The Last Broadcast,1998,1998-10-23,"Horror, Mystery",86,USA,English,...,46.0,1998,$,900,900,1.244111e+03,True,10,10,oct
4,54302,tt1127855,Ri-teon,Ri-teon,2007,2007-08-08,"Drama, Thriller",113,South Korea,Korean,...,11.0,2007,$,1000,1000,4.568780e+05,True,8,8,aug


In [39]:
#save the result to csv just in case
df1.to_csv('data/big_filtered_mths_05-31.csv')

In [41]:
#df['language']=df['language'].apply(lambda x: x.replace(',', '') if x==x else '')
cvt=CountVectorizer()
month_ary=cvt.fit_transform(df1['month_abbr']).toarray()
# feature_names=[f'language_{language}' for cvt.get_feature_names()
# feature_names=[]
month_df=pd.DataFrame(month_ary, columns=[f'month_{number}' for number in cvt.get_feature_names()])
month_df.head()

,month_apr,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep
0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,0


In [44]:
month_df.to_csv('month_ary.csv')